hello (Note : driving licence , car available)

# Setup

In [6]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res

from biogeme.expressions import Derive

In [7]:
# Read the data
# Read the data
df = pd.read_csv('lpmc07.dat', sep='\t')

# add a new column to original pandas data frame; this is to be used when computing elasticities
df['cost_driving'] = df['cost_driving_fuel'] + df['cost_driving_ccharge']
df['dur_pt'] = df['dur_pt_access'] + df['dur_pt_rail']+df['dur_pt_bus']+ df['dur_pt_int']

df.head()

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent,cost_driving,dur_pt
0,9,0,2,3,4,3,1,3,0.0,1,...,0.221111,0.0,0,0.222222,0.0,0.77,0.0,0.097500,0.77,0.426667
1,11,1,0,1,4,3,1,5,0.0,1,...,0.122222,0.0,0,0.132222,0.0,0.50,0.0,0.065126,0.50,0.363611
2,20,5,1,0,4,3,1,5,0.0,1,...,0.062222,0.0,0,0.117222,0.0,0.41,0.0,0.097156,0.41,0.443889
3,31,8,1,5,1,3,1,1,1.0,1,...,0.038056,0.0,0,0.055000,1.5,0.14,0.0,0.075758,0.14,0.139167
4,97,19,3,1,3,3,1,5,0.0,1,...,0.038333,0.2,2,0.716111,0.0,2.58,0.0,0.499612,2.58,0.878056


In [8]:
database = db.Database('lpmc07', df)

Defining the useful variables

In [9]:
#time variables
dur_walking = Variable('dur_walking')

dur_cycling = Variable('dur_cycling')

dur_driving = Variable('dur_driving')

dur_pt_access = Variable('dur_pt_access')
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int')

#cost variables
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')
cost_driving_ccharge = Variable('cost_driving_ccharge')

#choice
travel_mode = Variable('travel_mode')

#driving availability
driving_license = Variable('driving_license') 
car_ownership = Variable('car_ownership')

#female
female = Variable('female')

age = Variable('age')

#pt interchange
pt_interchanges = Variable('pt_interchanges')

driving_traffic_percent = Variable('driving_traffic_percent')

distance = Variable('distance')

trip_id = Variable('trip_id')

dur_pt = Variable('dur_pt')
cost_driving = Variable('cost_driving')

In [10]:
#dur_pt = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int
#cost_driving = cost_driving_fuel + cost_driving_ccharge

# model 0

In [6]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_driving = Beta('asc_driving', 0, None, None, 0)

beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

In [7]:
Opt0_walk = (
    beta_time*dur_walking
)
Opt0_cycle = (
    asc_cycle + beta_time*dur_cycling
)
Opt0_pt = (
    asc_pt + beta_time*dur_pt + beta_cost*cost_transit
)
Opt0_driv = (
    asc_driving + beta_time*dur_driving + beta_cost*cost_driving
)

In [8]:
print('car_ownership')

car_ownership


In [9]:
V0 = {1: Opt0_walk, 2: Opt0_cycle, 3: Opt0_pt, 4: Opt0_driv}


In [10]:
logprob0 = models.loglogit(V0, None, travel_mode) 

In [11]:
biogeme0 = bio.BIOGEME(database, logprob0)
biogeme0.modelName = 'logit_lpmc07_base'

In [12]:
results0 = biogeme0.estimate()

[11:15:06] < Warning >   Cannot read file __logit_lpmc07_base.iter. Statement is ignored.


In [13]:
print(results0.printGeneralStatistics())

Number of estimated parameters:	5
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4668.764
Likelihood ratio test for the init. model:	4525.416
Rho-square for the init. model:	0.326
Rho-square-bar for the init. model:	0.326
Akaike Information Criterion:	9347.528
Bayesian Information Criterion:	9380.114
Final gradient norm:	7.3760E-04
Nbr of threads:	16



In [14]:
results0.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.671670,0.107304,-34.217362,0.000000e+00
asc_driving,-1.104164,0.083172,-13.275731,0.000000e+00
asc_pt,-0.427631,0.055869,-7.654184,1.953993e-14
beta_cost,-0.151331,0.013355,-11.331599,0.000000e+00
beta_time,-5.129646,0.210478,-24.371452,0.000000e+00


# model 1
(we chose time)

In [11]:
asc_cycle1 = Beta('asc_cycle1', 0, None, None, 0)
asc_pt1 = Beta('asc_pt1', 0, None, None, 0)
asc_driving1 = Beta('asc_driving1', 0, None, None, 0)

beta_cost1 = Beta('beta_cost1', 0, None, None, 0)
beta_time_walk1 = Beta('beta_time_walk1', 0, None, None, 0)
beta_time_cycl1 = Beta('beta_time_cycl1', 0, None, None, 0)
beta_time_pt1 = Beta('beta_time_pt1', 0, None, None, 0)
beta_time_driv1 = Beta('beta_time_driv1', 0, None, None, 0)

In [12]:
Opt1_walk = (
    beta_time_walk1*dur_walking
)
Opt1_cycle = (
    asc_cycle1 + beta_time_cycl1*dur_cycling
)
Opt1_pt = (
    asc_pt1 + beta_time_pt1*dur_pt + beta_cost1*cost_transit
)
Opt1_driv = (
    asc_driving1 + beta_time_driv1*dur_driving + beta_cost1*cost_driving
)

In [13]:
V1 = {1: Opt1_walk, 2: Opt1_cycle, 3: Opt1_pt, 4: Opt1_driv}

In [14]:
logprob1 = models.loglogit(V1, None, travel_mode) 

In [15]:
biogeme1 = bio.BIOGEME(database, logprob1)
biogeme1.modelName = 'logit_lpmc07_as'

In [16]:
results1 = biogeme1.estimate()

[10:59:27] < Warning >   Cannot read file __logit_lpmc07_as.iter. Statement is ignored.


In [17]:
print(results1.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4361.567
Likelihood ratio test for the init. model:	5139.809
Rho-square for the init. model:	0.371
Rho-square-bar for the init. model:	0.37
Akaike Information Criterion:	8739.134
Bayesian Information Criterion:	8791.272
Final gradient norm:	8.3271E-03
Nbr of threads:	16



In [22]:
results1.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle1,-4.743495,0.210609,-22.522778,0.0
asc_driving1,-1.865705,0.157256,-11.864122,0.0
asc_pt1,-2.311875,0.157262,-14.700747,0.0
beta_cost1,-0.131324,0.014362,-9.144126,0.0
beta_time_cycl1,-4.880140,0.424974,-11.483382,0.0
beta_time_driv1,-6.113741,0.373771,-16.356896,0.0
beta_time_pt1,-3.270473,0.239894,-13.632966,0.0
beta_time_walk1,-8.208049,0.479568,-17.115501,0.0


Comparison between model 1 and model 0 performing a likelihood ratio test because model 1 is a restricted version of model 0. H0 is "model 0 is the true model".

In [23]:
general_statistics0 = results0.getGeneralStatistics()
general_statistics1 = results1.getGeneralStatistics()
LR = general_statistics0['Final log likelihood'].value
LU = general_statistics1['Final log likelihood'].value
test = -2 * (LR - LU)
test

614.39387600593

In [24]:
KR = general_statistics0['Number of estimated parameters'].value
KU = general_statistics1['Number of estimated parameters'].value
degrees_of_freedom = KU - KR
degrees_of_freedom

3

In [25]:
from scipy.stats import chi2
threshold = chi2.ppf(.95, degrees_of_freedom)
threshold

7.814727903251179

In [26]:
results1.likelihood_ratio_test(results0, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=614.39387600593, threshold=7.814727903251179)

model 1 is (way) better 

# model 2

In [27]:
asc_cycle2 = Beta('asc_cycle2', 0, None, None, 0)
asc_pt2 = Beta('asc_pt2', 0, None, None, 0)
asc_driving2 = Beta('asc_driving2', 0, None, None, 0)

#beta_pt_interchanges2 = Beta('pt_interchanges2',0, None, None, 0)



#age_0_25 = 0 < age <=100
#age_25_50 = 102 < age <= 103
#age_50_100 = 103 < age <= 105

#beta_cost_0_25 = Beta('beta_cost_0_25', 0, None, None, 0)
#beta_cost_25_50 = Beta('beta_cost_25_50', 0, None, None, 0)
#beta_cost_50_100 = Beta('beta_cost_50_100',0,None,None, 0)

#betas_piecewise = [beta_cost_0_25, beta_cost_25_50, beta_cost_50_100]

#cost_driving = df['cost_driving_ccharge'] + df['cost_driving_fuel']
#df['cost_driving'] = cost_driving

#cost_driving = Variable('cost_driving')

#piecewise_age = ((beta_cost_0_25*age_0_25)+(beta_cost_25_50*age_25_50)+(beta_cost_50_100*age_50_100))

#thresholds = [0, 20,50,60]

#thresholds = [0,18,60,None]

thresholds = [0,18,65,None]

beta_cost_youngold = Beta('beta_cost_youngold', 0, None, None, 0)
beta_cost_middleage = Beta('beta_cost_middleage', 0, None, None, 0)
#beta_cost_50_100 = Beta('beta_cost_50_100',0,None,None,0)

betas_piecewise = [beta_cost_youngold, beta_cost_middleage, beta_cost_youngold]


piecewise_age = models.piecewiseFormula(
    'age', thresholds, betas_piecewise
)

beta_traffic = Beta('beta_traffic',0,None,None,0)



beta_time_walk2 = Beta('beta_time_walk2', 0, None, None, 0)
beta_time_cycl2 = Beta('beta_time_cycl2', 0, None, None, 0)
beta_time_pt2 = Beta('beta_time_pt2', 0, None, None, 0)
beta_time_driv2 = Beta('beta_time_driv2', 0, None, None, 0)

In [28]:
Opt2_walk = (
    beta_time_walk2*dur_walking
)
Opt2_cycle = (
    asc_cycle2 + beta_time_cycl2*dur_cycling
)
Opt2_pt = (
    asc_pt2 + beta_time_pt2*dur_pt + piecewise_age*cost_transit/age 
)
Opt2_driv = (
    asc_driving2 + beta_time_driv2*dur_driving + piecewise_age*cost_driving/age + beta_traffic*driving_traffic_percent
)

In [29]:
V2 = {1: Opt2_walk, 2: Opt2_cycle, 3: Opt2_pt, 4: Opt2_driv}

In [30]:
logprob2 = models.loglogit(V2, None, travel_mode) 

In [31]:
biogeme2 = bio.BIOGEME(database, logprob2)
biogeme2.modelName = 'logit_lpmc07_2'

In [32]:
results2 = biogeme2.estimate()

[11:15:11] < Warning >   Cannot read file __logit_lpmc07_2.iter. Statement is ignored.


In [33]:
print(results2.printGeneralStatistics())

Number of estimated parameters:	10
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4272.921
Likelihood ratio test for the init. model:	5317.101
Rho-square for the init. model:	0.384
Rho-square-bar for the init. model:	0.382
Akaike Information Criterion:	8565.843
Bayesian Information Criterion:	8631.015
Final gradient norm:	1.6733E-02
Nbr of threads:	16



In [34]:
results2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle2,-4.769377,0.213384,-22.351194,0.000000
asc_driving2,-1.357151,0.161688,-8.393631,0.000000
asc_pt2,-2.469546,0.161118,-15.327582,0.000000
beta_cost_middleage,-0.080101,0.047630,-1.681746,0.092618
beta_cost_youngold,-0.102830,0.037603,-2.734643,0.006245
beta_time_cycl2,-4.291779,0.422903,-10.148372,0.000000
beta_time_driv2,-4.109835,0.374498,-10.974263,0.000000
beta_time_pt2,-2.686530,0.234331,-11.464694,0.000000
beta_time_walk2,-8.062313,0.478669,-16.843198,0.000000
beta_traffic,-2.858280,0.229074,-12.477522,0.000000


we perform ll ratio

In [35]:
general_statistics1 = results1.getGeneralStatistics()
general_statistics2 = results2.getGeneralStatistics()
LR = general_statistics1['Final log likelihood'].value
LU = general_statistics2['Final log likelihood'].value
test = -2 * (LR - LU)
test

177.29153031231726

In [36]:
KR = general_statistics1['Number of estimated parameters'].value
KU = general_statistics2['Number of estimated parameters'].value
degrees_of_freedom = KU - KR
degrees_of_freedom

2

In [37]:
threshold = chi2.ppf(.95, degrees_of_freedom)
threshold

5.991464547107979

# Model 3

In [38]:
asc_cycle3 = Beta('asc_cycle3', 0, None, None, 0)
asc_pt3 = Beta('asc_pt3', 0, None, None, 0)
asc_driving3 = Beta('asc_driving3', 0, None, None, 0)

#beta_pt_interchanges3 = Beta('pt_interchanges3',0, None, None, 0)

#thresholds = [0, 25, 50, 100]

#beta_cost_0_25 = Beta('beta_cost_0_25', 0, None, None, 0)
#beta_cost_25_50 = Beta('beta_cost_25_50', 0, None, None, 0)
#beta_cost_50_100 = Beta('beta_cost_50_100',0,None,None,0)

#betas_piecewise = [beta_cost_0_25, beta_cost_25_50, beta_cost_50_100]

#cost_driving = df['cost_driving_ccharge'] + df['cost_driving_fuel']
#df['cost_driving'] = cost_driving

#cost_driving = Variable('cost_driving')

#piecewise_age = models.piecewiseFormula(
#    'age', thresholds, betas_piecewise
#)

##thresholds = [0,18,50]

##beta_cost_0_25 = Beta('beta_cost_0_25', 0, None, None, 0)
##beta_cost_25_50 = Beta('beta_cost_25_50', 0, None, None, 0)
##beta_cost_50_100 = Beta('beta_cost_50_100',0,None,None,0)

##betas_piecewise = [beta_cost_0_25, beta_cost_25_50]#, beta_cost_50_100]

thresholds = [0,18,65,None]

beta_cost_youngold = Beta('beta_cost_youngold', 0, None, None, 0)
beta_cost_middleage = Beta('beta_cost_middleage', 0, None, None, 0)
#beta_cost_50_100 = Beta('beta_cost_50_100',0,None,None,0)

betas_piecewise = [beta_cost_youngold, beta_cost_middleage, beta_cost_youngold]


piecewise_age = models.piecewiseFormula(
    'age', thresholds, betas_piecewise
)

beta_traffic = Beta('beta_traffic',0,None,None,0)



beta_time_walk3 = Beta('beta_time_walk3', 0, None, None, 0)
beta_time_cycl3 = Beta('beta_time_cycl3', 0, None, None, 0)
beta_time_pt3 = Beta('beta_time_pt3', 0, None, None, 0)
beta_time_driv3 = Beta('beta_time_driv3', 0, None, None, 0)

lambda_cox = Beta('lambda_cox', 0, None, None, 0)

In [39]:
Opt3_walk = (    
    beta_time_walk3*models.boxcox(dur_walking, lambda_cox)
)
Opt3_cycle = (    
    asc_cycle3 + beta_time_cycl3*models.boxcox(dur_cycling, lambda_cox)
)
Opt3_pt = (    
    asc_pt3 + beta_time_pt3*models.boxcox(dur_pt, lambda_cox) + piecewise_age*cost_transit/age #+ beta_pt_interchanges3*pt_interchanges
)
Opt3_driv = (    
    asc_driving3 + beta_time_driv3*models.boxcox(dur_driving, lambda_cox) + piecewise_age*cost_driving/age + beta_traffic*driving_traffic_percent
)

In [40]:
V3 = {1: Opt3_walk, 2: Opt3_cycle, 3: Opt3_pt, 4: Opt3_driv}

In [41]:
logprob3 = models.loglogit(V3, None, travel_mode) 

In [42]:
biogeme3 = bio.BIOGEME(database, logprob3)
biogeme3.modelName = 'logit_lpmc07_3'

In [43]:
results3 = biogeme3.estimate()
print(results3.printGeneralStatistics())

[11:15:17] < Warning >   Cannot read file __logit_lpmc07_3.iter. Statement is ignored.
Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4224.293
Likelihood ratio test for the init. model:	5414.358
Rho-square for the init. model:	0.391
Rho-square-bar for the init. model:	0.389
Akaike Information Criterion:	8470.585
Bayesian Information Criterion:	8542.275
Final gradient norm:	1.5017E-02
Nbr of threads:	16



In [44]:
results3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle3,-1.583609,0.261759,-6.049883,1.449509e-09
asc_driving3,2.007417,0.276827,7.251523,4.121148e-13
asc_pt3,2.096810,0.192897,10.870072,0.000000e+00
beta_cost_middleage,-0.080281,0.046440,-1.728710,8.386108e-02
beta_cost_youngold,-0.104890,0.036701,-2.857933,4.264107e-03
beta_time_cycl3,-2.771042,0.297238,-9.322635,0.000000e+00
beta_time_driv3,-2.521645,0.299948,-8.406939,0.000000e+00
beta_time_pt3,-2.155863,0.203373,-10.600513,0.000000e+00
beta_time_walk3,-5.374291,0.372863,-14.413584,0.000000e+00
beta_traffic,-2.530745,0.236234,-10.712891,0.000000e+00


In [53]:
t_test_lambda = (0.379196-1)/0.0703231
t_test_lambda

-8.827881592250627

In [45]:

general_statistics2 = results2.getGeneralStatistics()
general_statistics3 = results3.getGeneralStatistics()
LR = general_statistics2['Final log likelihood'].value
LU = general_statistics3['Final log likelihood'].value
test = -2 * (LR - LU)
test

97.25716635595563

In [54]:
KR = general_statistics2['Number of estimated parameters'].value
KU = general_statistics3['Number of estimated parameters'].value
degrees_of_freedom = KU - KR
degrees_of_freedom

1

In [58]:
threshold = chi2.ppf(.95, 1)
threshold

3.841458820694124

# Model 4
The nests are defined as
- A) low concentration and low accident risk: walking and public transport.high concentration and high accident risk: driving and cycling
- B) high concentration and high accident risk: driving and cycling

In [48]:
asc_cycle4 = Beta('asc_cycle4', 0, None, None, 0)
asc_pt4 = Beta('asc_pt4', 0, None, None, 0)
asc_driving4 = Beta('asc_driving4', 0, None, None, 0)

#beta_pt_interchanges3 = Beta('pt_interchanges3',0, None, None, 0)

#thresholds = [0, 25, 50, 100]

#beta_cost_0_25 = Beta('beta_cost_0_25', 0, None, None, 0)
#beta_cost_25_50 = Beta('beta_cost_25_50', 0, None, None, 0)
#beta_cost_50_100 = Beta('beta_cost_50_100',0,None,None,0)

#betas_piecewise = [beta_cost_0_25, beta_cost_25_50, beta_cost_50_100]

#cost_driving = df['cost_driving_ccharge'] + df['cost_driving_fuel']
#df['cost_driving'] = cost_driving

#cost_driving = Variable('cost_driving')

#piecewise_age = models.piecewiseFormula(
#    'age', thresholds, betas_piecewise
#)

#thresholds = [0,18,50]

#beta_cost_0_25 = Beta('beta_cost_0_25', 0, None, None, 0)
#beta_cost_25_50 = Beta('beta_cost_25_50', 0, None, None, 0)
#beta_cost_50_100 = Beta('beta_cost_50_100',0,None,None,0)

thresholds = [0,18,65,None]

beta_cost_youngold = Beta('beta_cost_youngold', 0, None, None, 0)
beta_cost_middleage = Beta('beta_cost_middleage', 0, None, None, 0)
#beta_cost_50_100 = Beta('beta_cost_50_100',0,None,None,0)

betas_piecewise = [beta_cost_youngold, beta_cost_middleage, beta_cost_youngold]


piecewise_age = models.piecewiseFormula(
    'age', thresholds, betas_piecewise
)

beta_traffic = Beta('beta_traffic',0,None,None,0)



beta_time_walk4 = Beta('beta_time_walk4', 0, None, None, 0)
beta_time_cycl4 = Beta('beta_time_cycl4', 0, None, None, 0)
beta_time_pt4 = Beta('beta_time_pt4', 0, None, None, 0)
beta_time_driv4 = Beta('beta_time_driv4', 0, None, None, 0)

lambda_cox = Beta('lambda_cox', 0, None, None, 0)


Opt4_walk = (    
    beta_time_walk4*models.boxcox(dur_walking, lambda_cox)
)
Opt4_cycle = (    
    asc_cycle4 + beta_time_cycl4*models.boxcox(dur_cycling, lambda_cox)
)
Opt4_pt = (    
    asc_pt4 + beta_time_pt4*models.boxcox(dur_pt, lambda_cox) + piecewise_age*cost_transit/age #+ beta_pt_interchanges3*pt_interchanges
)
Opt4_driv = (    
    asc_driving4 + beta_time_driv4*models.boxcox(dur_driving, lambda_cox) + piecewise_age*cost_driving/age + beta_traffic*driving_traffic_percent
)

V4 = {1: Opt4_walk, 2: Opt4_cycle, 3: Opt4_pt, 4: Opt4_driv}

# Define the nests
mu_a = Beta('mu_a', 1, 0, None, 0)
mu_b = Beta('mu_b', 1, 0, None, 0)
nest_a = mu_a, [1, 3] # 1 and 3 refers to walk and pt as in the dictionary V1
nest_b = mu_b, [2, 4]
nests = nest_a, nest_b

#availability

logprob4 = models.lognested(V4, None, nests, travel_mode)

biogeme4 = bio.BIOGEME(database, logprob4)
biogeme4.modelName = 'logit_lpmc07_as'

results4 = biogeme4.estimate()

print(results4.printGeneralStatistics())

results4.getEstimatedParameters()

Number of estimated parameters:	13
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4220.637
Likelihood ratio test for the init. model:	5421.67
Rho-square for the init. model:	0.391
Rho-square-bar for the init. model:	0.389
Akaike Information Criterion:	8467.273
Bayesian Information Criterion:	8551.997
Final gradient norm:	2.6582E-02
Nbr of threads:	16



,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle4,-1.627140,0.370227,-4.394982,1.107818e-05
asc_driving4,1.622294,0.300275,5.402689,6.564905e-08
asc_pt4,1.647707,0.245517,6.711181,1.930567e-11
beta_cost_middleage,-0.075355,0.043291,-1.740688,8.173827e-02
beta_cost_youngold,-0.107561,0.034985,-3.074511,2.108483e-03
beta_time_cycl4,-2.678963,0.290576,-9.219496,0.000000e+00
beta_time_driv4,-2.409871,0.305609,-7.885460,3.108624e-15
beta_time_pt4,-2.166825,0.212811,-10.181926,0.000000e+00
beta_time_walk4,-4.873209,0.376794,-12.933348,0.000000e+00
beta_traffic,-2.462150,0.233260,-10.555405,0.000000e+00


In [59]:
t_test_lambda = (0.382546-1)/0.073875
t_test_lambda

-8.358091370558375

In [60]:
t_test_mua = (0.208714)/0.119783
t_test_mua

1.7424342352420628

In [61]:
t_test_mub = (0.172845)/0.113761
t_test_mub

1.5193695554715587

test

In [49]:
general_statistics3 = results3.getGeneralStatistics()
general_statistics4 = results4.getGeneralStatistics()
LR = general_statistics3['Final log likelihood'].value
LU = general_statistics4['Final log likelihood'].value
test = -2 * (LR - LU)
test

7.312058576355412

In [50]:
KR = general_statistics3['Number of estimated parameters'].value
KU = general_statistics4['Number of estimated parameters'].value
degrees_of_freedom = KU - KR
degrees_of_freedom

2

In [51]:
threshold = chi2.ppf(.95, degrees_of_freedom)
threshold

5.991464547107979

# Market shares

In [53]:
census = {
    'male_41_more':  1633263,
    'male_40_less':  2676249,
    'female_41_more':  1765143,
    'female_40_less':  2599058,
}

total = sum(census.values())
total

8673713

In [54]:
filters = {
    'male_41_more': (df.age > 40) & (df.female == 0),
    'male_40_less': (df.age <= 40) & (df.female == 0),
    'female_41_more': (df.age > 40) & (df.female == 1),
    'female_40_less': (df.age <= 40) & (df.female == 1),
}

sample_segments = {
    k: v.sum() for k, v in filters.items()
}
sample_segments

{'male_41_more': 1075,
 'male_40_less': 1270,
 'female_41_more': 1196,
 'female_40_less': 1457}

In [55]:
total_sample = sum(sample_segments.values())
total_sample

4998

In [56]:
weights = {
    k: census[k] * total_sample / (v * total) 
    for k, v in sample_segments.items()
}
weights

{'male_41_more': 0.8754651513355407,
 'male_40_less': 1.2142664647161419,
 'female_41_more': 0.850432637527414,
 'female_40_less': 1.0278923250803063}

In [57]:
for k, f in filters.items():
    df.loc[f, 'Weight'] = weights[k]


In [58]:
from collections import namedtuple
IndicatorTuple = namedtuple('IndicatorTuple', 'value lower upper') 

In [59]:
logprob4 = models.loglogit(V4, None, travel_mode) 

biogeme4 = bio.BIOGEME(database, logprob4)
biogeme4.modelName = 'logit_lpmc07_as'

results = biogeme4.estimate(bootstrap=100, recycle=False)

100%|██████████| 100/100 [00:14<00:00,  6.73it/s]


In [60]:
Weight = Variable('Weight')

In [61]:
def market_share(utility):
    
    prob_WALK = models.logit(utility, None, 1)
    prob_CYCLE = models.logit(utility, None, 2)
    prob_PT = models.logit(utility, None, 3)
    prob_DRIVE = models.logit(utility, None, 4)

    simulate = {
        'Weight': Weight,
        'Prob. walk': prob_WALK,
        'Prob. cycle': prob_CYCLE,
        'Prob. PT': prob_PT,
        'Prob. drive' : prob_DRIVE,
    }
    biosim = bio.BIOGEME(database, simulate)
    simulated_values = biosim.simulate(results.getBetaValues())

    # We also calculate confidence intervals for the calculated quantities

    betas = biogeme4.freeBetaNames()
    b = results.getBetasForSensitivityAnalysis(betas)
    left, right = biosim.confidenceIntervals(b, 0.9)

    # Market shares are calculated using the weighted mean of the individual probabilities

    ## Alternative walk

    simulated_values['Weighted prob. walk'] = (
        simulated_values['Weight'] * simulated_values['Prob. walk']
    )
    left['Weighted prob. walk'] = left['Weight'] * left['Prob. walk']
    right['Weighted prob. walk'] = (
                        right['Weight'] * right['Prob. walk']
    )

    marketShare_walk = simulated_values['Weighted prob. walk'].mean()
    marketShare_walk_left = left['Weighted prob. walk'].mean()
    marketShare_walk_right = right['Weighted prob. walk'].mean()
    
        ## Alternative cycle

    simulated_values['Weighted prob. cycle'] = (
        simulated_values['Weight'] * simulated_values['Prob. cycle']
    )
    left['Weighted prob. cycle'] = left['Weight'] * left['Prob. cycle']
    right['Weighted prob. cycle'] = right['Weight'] * right['Prob. cycle']

    marketShare_cycle = simulated_values['Weighted prob. cycle'].mean()
    marketShare_cycle_left = left['Weighted prob. cycle'].mean()
    marketShare_cycle_right = right['Weighted prob. cycle'].mean()

    ## Alternative public transportation

    simulated_values['Weighted prob. PT'] = (
        simulated_values['Weight'] * simulated_values['Prob. PT']
    )
    left['Weighted prob. PT'] = left['Weight'] * left['Prob. PT']
    right['Weighted prob. PT'] = right['Weight'] * right['Prob. PT']

    marketShare_PT = simulated_values['Weighted prob. PT'].mean()
    marketShare_PT_left = left['Weighted prob. PT'].mean()
    marketShare_PT_right = right['Weighted prob. PT'].mean()

    ## Alternative drive

    simulated_values['Weighted prob. drive'] = (
        simulated_values['Weight'] * simulated_values['Prob. drive']
    )
    left['Weighted prob. drive'] = left['Weight'] * left['Prob. drive']
    right['Weighted prob. drive'] = right['Weight'] * right['Prob. drive']

    marketShare_drive = simulated_values['Weighted prob. drive'].mean()
    marketShare_drive_left = left['Weighted prob. drive'].mean()
    marketShare_drive_right = right['Weighted prob. drive'].mean()
    

    return {
            'Walk': IndicatorTuple(
                value=marketShare_walk, 
                lower=marketShare_walk_left, 
                upper=marketShare_walk_right
            ),
            'Cycle': IndicatorTuple(
                value=marketShare_cycle, 
                lower=marketShare_cycle_left, 
                upper=marketShare_cycle_right
            ),
            'Public transportation': IndicatorTuple(
                value=marketShare_PT, 
                lower=marketShare_PT_left, 
                upper=marketShare_PT_right
            ),
            'Drive': IndicatorTuple(
                value=marketShare_drive, 
                lower=marketShare_drive_left, 
                upper=marketShare_drive_right
            )
    }

In [62]:
ms = market_share(V4)
for k, v in ms.items():
    print(
        f'Market share for {k}: {100*v.value:.1f}% '
        f'[{100*v.lower:.1f}%, '
        f'{100*v.upper:.1f}%]'
    )

[19:00:19] < Warning >   The chosen alternative [`1.0`] is not available for the following observations (rownumber[choice]): 4[1.0]-20[1.0]-24[1.0]-25[1.0]-31[1.0]-32[1.0]-33[1.0]-36[1.0]-37[1.0]-38[1.0]-48[1.0]-50[1.0]-52[1....
[19:00:19] < Warning >   The chosen alternative [`1.0`] is not available for the following observations (rownumber[choice]): 4[1.0]-20[1.0]-24[1.0]-25[1.0]-31[1.0]-32[1.0]-33[1.0]-36[1.0]-37[1.0]-38[1.0]-48[1.0]-50[1.0]-52[1....
[19:00:19] < Warning >   The chosen alternative [`1.0`] is not available for the following observations (rownumber[choice]): 4[1.0]-20[1.0]-24[1.0]-25[1.0]-31[1.0]-32[1.0]-33[1.0]-36[1.0]-37[1.0]-38[1.0]-48[1.0]-50[1.0]-52[1....
[19:00:19] < Warning >   The chosen alternative [`1.0`] is not available for the following observations (rownumber[choice]): 4[1.0]-20[1.0]-24[1.0]-25[1.0]-31[1.0]-32[1.0]-33[1.0]-36[1.0]-37[1.0]-38[1.0]-48[1.0]-50[1.0]-52[1....
[19:00:20] < Warning >   The chosen alternative [`1.0`] is not available for the fol

scenario (i) : increase of car cost by 15%, market shares

In [63]:
asc_cycle4 = Beta('asc_cycle4', 0, None, None, 0)
asc_pt4 = Beta('asc_pt4', 0, None, None, 0)
asc_driving4 = Beta('asc_driving4', 0, None, None, 0)

thresholds = [0,18,65,None]

beta_cost_youngold = Beta('beta_cost_youngold', 0, None, None, 0)
beta_cost_middleage = Beta('beta_cost_middleage', 0, None, None, 0)
#beta_cost_50_100 = Beta('beta_cost_50_100',0,None,None,0)

betas_piecewise = [beta_cost_youngold, beta_cost_middleage, beta_cost_youngold]

piecewise_age = models.piecewiseFormula(
    'age', thresholds, betas_piecewise
)

beta_traffic = Beta('beta_traffic',0,None,None,0)

beta_time_walk4 = Beta('beta_time_walk4', 0, None, None, 0)
beta_time_cycl4 = Beta('beta_time_cycl4', 0, None, None, 0)
beta_time_pt4 = Beta('beta_time_pt4', 0, None, None, 0)
beta_time_driv4 = Beta('beta_time_driv4', 0, None, None, 0)

lambda_cox = Beta('lambda_cox', 0, None, None, 0)

Opt5_walk = (    
    beta_time_walk4*models.boxcox(dur_walking, lambda_cox)
)
Opt5_cycle = (    
    asc_cycle4 + beta_time_cycl4*models.boxcox(dur_cycling, lambda_cox)
)
Opt5_pt = (    
    asc_pt4 + beta_time_pt4*models.boxcox(dur_pt, lambda_cox) 
    + piecewise_age*cost_transit/age #+ beta_pt_interchanges3*pt_interchanges
)
Opt5_driv = (    
    asc_driving4 + beta_time_driv4*models.boxcox(dur_driving, lambda_cox) 
    + piecewise_age*cost_driving*1.15/age + beta_traffic*driving_traffic_percent
)

V5 = {1: Opt5_walk, 2: Opt5_cycle, 3: Opt5_pt, 4: Opt5_driv}

####

# Define the nests
mu_a = Beta('mu_a', 1, 0, None, 0)
mu_b = Beta('mu_b', 1, 0, None, 0)
nest_a = mu_a, [1, 3] # 1 and 3 refers to walk and pt as in the dictionary V1
nest_b = mu_b, [2, 4]
nests = nest_a, nest_b
#availability
av = None
logprob4 = models.lognested(V5, None, nests, travel_mode)
biogeme4 = bio.BIOGEME(database, logprob4)
biogeme4.modelName = 'logit_lpmc07_as'
results4 = biogeme4.estimate()

####

prob_WALK1 = models.logit(V5, None, 1)    #QUESTION logit?
prob_CYCLE1 = models.logit(V5, None, 2)
prob_PT1 = models.logit(V5, None, 3)
prob_DRIVE1 = models.logit(V5, None, 4)

simulate1 = {
    'Weight': Weight,
    'Prob. walk': prob_WALK1,
    'Prob. cycle': prob_CYCLE1,
    'Prob. PT': prob_PT1,
    'Prob. drive' : prob_DRIVE1,
}
biosim = bio.BIOGEME(database, simulate1)
simulated_values1 = biosim.simulate(results.getBetaValues())

####

## Alternative walk
simulated_values1['Weighted prob. walk'] = (
    simulated_values1['Weight'] * simulated_values1['Prob. walk']
)
marketShare_walk1 = simulated_values1['Weighted prob. walk'].mean()
    
## Alternative cycle
simulated_values1['Weighted prob. cycle'] = (
    simulated_values1['Weight'] * simulated_values1['Prob. cycle']
)
marketShare_cycle1 = simulated_values1['Weighted prob. cycle'].mean()

## Alternative public transportation
simulated_values1['Weighted prob. PT'] = (
    simulated_values1['Weight'] * simulated_values1['Prob. PT']
)
marketShare_PT1 = simulated_values1['Weighted prob. PT'].mean()

## Alternative drive
simulated_values1['Weighted prob. drive'] = (
    simulated_values1['Weight'] * simulated_values1['Prob. drive']
)
marketShare_drive1 = simulated_values1['Weighted prob. drive'].mean()

####

print(f'Market share for walk: {100*marketShare_walk1:.1f}%')
print(f'Market share for cycle: {100*marketShare_cycle1:.1f}%')
print(f'Market share for PT: {100*marketShare_PT1:.1f}%')
print(f'Market share for drive: {100*marketShare_drive1:.1f}%')

Market share for walk: 16.6%
Market share for cycle: 3.0%
Market share for PT: 36.1%
Market share for drive: 44.3%


scenario (ii) : decrease of PT cost by 15%, market shares

In [64]:
asc_cycle4 = Beta('asc_cycle4', 0, None, None, 0)
asc_pt4 = Beta('asc_pt4', 0, None, None, 0)
asc_driving4 = Beta('asc_driving4', 0, None, None, 0)

thresholds = [0,18,65,None]

beta_cost_youngold = Beta('beta_cost_youngold', 0, None, None, 0)
beta_cost_middleage = Beta('beta_cost_middleage', 0, None, None, 0)
#beta_cost_50_100 = Beta('beta_cost_50_100',0,None,None,0)

betas_piecewise = [beta_cost_youngold, beta_cost_middleage, beta_cost_youngold]

piecewise_age = models.piecewiseFormula(
    'age', thresholds, betas_piecewise
)

beta_traffic = Beta('beta_traffic',0,None,None,0)

beta_time_walk4 = Beta('beta_time_walk4', 0, None, None, 0)
beta_time_cycl4 = Beta('beta_time_cycl4', 0, None, None, 0)
beta_time_pt4 = Beta('beta_time_pt4', 0, None, None, 0)
beta_time_driv4 = Beta('beta_time_driv4', 0, None, None, 0)

lambda_cox = Beta('lambda_cox', 0, None, None, 0)

Opt6_walk = (    
    beta_time_walk4*models.boxcox(dur_walking, lambda_cox)
)
Opt6_cycle = (    
    asc_cycle4 + beta_time_cycl4*models.boxcox(dur_cycling, lambda_cox)
)
Opt6_pt = (    
    asc_pt4 + beta_time_pt4*models.boxcox(dur_pt, lambda_cox) 
    + piecewise_age*cost_transit*0.85/age #+ beta_pt_interchanges3*pt_interchanges
)
Opt6_driv = (    
    asc_driving4 + beta_time_driv4*models.boxcox(dur_driving, lambda_cox) 
    + piecewise_age*cost_driving/age + beta_traffic*driving_traffic_percent
)

V6 = {1: Opt6_walk, 2: Opt6_cycle, 3: Opt6_pt, 4: Opt6_driv}

####

# Define the nests
mu_a = Beta('mu_a', 1, 0, None, 0)
mu_b = Beta('mu_b', 1, 0, None, 0)
nest_a = mu_a, [1, 3] # 1 and 3 refers to walk and pt as in the dictionary V1
nest_b = mu_b, [2, 4]
nests = nest_a, nest_b
#availability
av = None
logprob4 = models.lognested(V5, None, nests, travel_mode)
biogeme4 = bio.BIOGEME(database, logprob4)
biogeme4.modelName = 'logit_lpmc07_as'
results4 = biogeme4.estimate()

####

prob_WALK2 = models.logit(V6, None, 1)    #QUESTION logit?
prob_CYCLE2 = models.logit(V6, None, 2)
prob_PT2 = models.logit(V6, None, 3)
prob_DRIVE2 = models.logit(V6, None, 4)

simulate2 = {
    'Weight': Weight,
    'Prob. walk': prob_WALK2,
    'Prob. cycle': prob_CYCLE2,
    'Prob. PT': prob_PT2,
    'Prob. drive' : prob_DRIVE2,
}
biosim = bio.BIOGEME(database, simulate2)
simulated_values2 = biosim.simulate(results.getBetaValues())

####

## Alternative walk
simulated_values2['Weighted prob. walk'] = (
    simulated_values2['Weight'] * simulated_values2['Prob. walk']
)
marketShare_walk2 = simulated_values2['Weighted prob. walk'].mean()
    
## Alternative cycle
simulated_values2['Weighted prob. cycle'] = (
    simulated_values2['Weight'] * simulated_values2['Prob. cycle']
)
marketShare_cycle2 = simulated_values2['Weighted prob. cycle'].mean()

## Alternative public transportation
simulated_values2['Weighted prob. PT'] = (
    simulated_values2['Weight'] * simulated_values2['Prob. PT']
)
marketShare_PT2 = simulated_values2['Weighted prob. PT'].mean()

## Alternative drive
simulated_values2['Weighted prob. drive'] = (
    simulated_values2['Weight'] * simulated_values2['Prob. drive']
)
marketShare_drive2 = simulated_values2['Weighted prob. drive'].mean()

####

print(f'Market share for walk: {100*marketShare_walk2:.1f}%')
print(f'Market share for cycle: {100*marketShare_cycle2:.1f}%')
print(f'Market share for PT: {100*marketShare_PT2:.1f}%')
print(f'Market share for drive: {100*marketShare_drive2:.1f}%')

Market share for walk: 16.5%
Market share for cycle: 3.0%
Market share for PT: 36.2%
Market share for drive: 44.3%


# VALUE OF TIME

In [68]:
# you need to run Model 4 before this

#retrieve data from the database and from results of Model 4
dataframe = database.data
betas = results4.getBetaValues()

beta_time_driv4 = betas["beta_time_driv4"]
beta_time_pt4 = betas["beta_time_pt4"]
beta_cost_youngold = betas["beta_cost_youngold"]
beta_cost_middleage = betas["beta_cost_middleage"]
lambda_cox = betas["lambda_cox"]

# prepare the data
# boolean list. if young_old_indexes[n]=True, then person n is younger than 18 or older than 61
young_old_indexes = np.any([dataframe["age"] <= 18, dataframe["age"] > 61], axis=0)
middleage_indexes = np.all([dataframe["age"] > 18, dataframe["age"] <= 61], axis=0) 

dur_driv_youngold = dataframe[young_old_indexes]["dur_driving"]
dur_driv_middleage = dataframe[middleage_indexes]["dur_driving"]
dur_pt_youngold = dataframe[young_old_indexes]["dur_pt_access"] + \
    dataframe[young_old_indexes]["dur_pt_rail"] + \
    dataframe[young_old_indexes]["dur_pt_bus"] + \
    dataframe[young_old_indexes]["dur_pt_int"]
dur_pt_middleage = dataframe[middleage_indexes]["dur_pt_access"] + \
    dataframe[middleage_indexes]["dur_pt_rail"] + \
    dataframe[middleage_indexes]["dur_pt_bus"] + \
    dataframe[middleage_indexes]["dur_pt_int"]

vot_driv_youngold = beta_time_driv4 / beta_cost_youngold * dur_driv_youngold ** (lambda_cox - 1)
vot_driv_middleage = beta_time_driv4 / beta_cost_middleage * dur_driv_middleage ** (lambda_cox - 1)

vot_pt_youngold = beta_time_pt4 / beta_cost_youngold * dur_pt_youngold ** (lambda_cox - 1)
vot_pt_middleage = beta_time_pt4 / beta_cost_middleage * dur_pt_middleage ** (lambda_cox - 1)

print("average value of time for the young or old who are driving (GBP / hour):")
print(np.mean(vot_driv_youngold))

print("\naverage value of time for the middleaged who are driving (GBP / hour):")
print(np.mean(vot_driv_middleage))

print("\naverage value of time for the young or old who use public transport (GBP / hour):")
print(np.mean(vot_pt_youngold))

print("\naverage value of time for the middleaged who use public trasnport (GBP / hour):")
print(np.mean(vot_pt_middleage))

print("\naverage value of time for anyone driving (GBP / hour):")
print(np.mean([np.mean(vot_driv_youngold), np.mean(vot_driv_middleage)]))

print("\naverage value of time for anyone using public transport (GBP / hour):")
print(np.mean([np.mean(vot_pt_youngold), np.mean(vot_pt_middleage)]))

average value of time for the young or old who are driving (GBP / hour):
67.54971196595159

average value of time for the middleaged who are driving (GBP / hour):
153.29789197599447

average value of time for the young or old who use public transport (GBP / hour):
40.208109218574634

average value of time for the middleaged who use public trasnport (GBP / hour):
95.49167939324131

average value of time for anyone driving (GBP / hour):
110.42380197097303

average value of time for anyone using public transport (GBP / hour):
67.84989430590798


# ELASTICITIES

In [65]:
# Normalize the weights
sumWeight = database.data['Weight'].sum()
numberOfRows = database.data.shape[0]
normalizedWeight = Weight * numberOfRows / sumWeight

In [66]:
normalizedWeight

((Weight * `4998.0`) / `4998.0`)

In [ ]:
logprob4 = models.loglogit(V4, None, travel_mode) 

biogeme4 = bio.BIOGEME(database, logprob4)
biogeme4.modelName = 'logit_lpmc07_as'

results = biogeme4.estimate(bootstrap=100, recycle=False)

In [67]:
def elasticity(V, av):
    
    prob_PT = models.logit(V, av, 3)
    prob_DRIVE = models.logit(V, av, 4)
    
    direct_elas_pt_cost = ( Derive(prob_PT, 'cost_transit') * cost_transit / prob_PT )    
    direct_elas_car_cost = ( Derive(prob_DRIVE, 'cost_driving') * cost_driving / prob_DRIVE )
        
    simulate = {
        'Weight': normalizedWeight,
        'Prob. public transportation': prob_PT,
        'direct_elas_pt_cost': direct_elas_pt_cost,
        'Prob. drive': prob_DRIVE,
        'direct_elas_drive_cost': direct_elas_car_cost,
    }
    
    biosim = bio.BIOGEME(database, simulate)
    simulated_values = biosim.simulate(results.getBetaValues())
    
    # aggregate elastiticities for public transportation
    
    simulated_values['denominator_pt'] = (
    simulated_values['Weight'] *
    simulated_values['Prob. public transportation'])
        
    simulated_values['numerator_pt_cost'] = (
    simulated_values['Weight'] *
    simulated_values['Prob. public transportation'] *
    simulated_values['direct_elas_pt_cost'])
    
    # aggregate elastiticities for driving
    
    simulated_values['denominator_drive'] = (
    simulated_values['Weight'] *
    simulated_values['Prob. drive'])
    
    simulated_values['numerator_drive_cost'] = (
    simulated_values['Weight'] *
    simulated_values['Prob. drive'] *
    simulated_values['direct_elas_drive_cost'])
    
    agg_elast_pt_cost = simulated_values['numerator_pt_cost'].sum() / simulated_values['denominator_pt'].sum()
    
    agg_elast_drive_cost = simulated_values['numerator_drive_cost'].sum() / simulated_values['denominator_drive'].sum()
        
    return simulated_values, agg_elast_pt_cost, agg_elast_drive_cost

simulated_values = elasticity(V4, av)
simulated_values

(        Weight  Prob. public transportation  direct_elas_pt_cost  Prob. drive  \
 0     1.027892                     0.280454             0.000000     0.700980   
 1     0.850433                     0.186586             0.000000     0.788966   
 2     0.850433                     0.132209             0.000000     0.802028   
 3     0.850433                     0.087917            -0.121429     0.308860   
 4     1.027892                     0.745305             0.000000     0.240850   
 ...        ...                          ...                  ...          ...   
 4995  1.027892                     0.366766            -0.090298     0.590144   
 4996  1.214266                     0.349819            -0.088126     0.622680   
 4997  0.875465                     0.099231            -0.120123     0.232398   
 4998  1.027892                     0.796139            -0.125339     0.187185   
 4999  1.214266                     0.230735            -0.108091     0.631365   
 
       direct_